In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# Aumenta o limite de exibição de colunas e linhas no pandas
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)


df_analises_original = pd.read_csv("data/dados_pns_2019_processado.csv")
dic = pd.read_csv("data/dic.csv")
#28s


# ajuste dic

In [33]:
# Renomeia colunas com nomes padrão
dic.columns = [col.strip() for col in dic.columns]

# Identifica nomes exatos das colunas que queremos preencher
col_codigo = [col for col in dic.columns if 'Código\nda\nvariável' in col][0]
col_quesito = [col for col in dic.columns if 'Unnamed: 4' in col][0]

# Preenche ambos com forward fill
dic[col_codigo] = dic[col_codigo].fillna(method='ffill')
dic[col_quesito] = dic[col_quesito].fillna(method='ffill')



/tmp/ipykernel_1119105/2849416733.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dic[col_codigo] = dic[col_codigo].fillna(method='ffill')
/tmp/ipykernel_1119105/2849416733.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dic[col_quesito] = dic[col_quesito].fillna(method='ffill')


In [44]:
resumo_dic = dic[['Unnamed: 4', 'Código\nda\nvariável', 'Categorias',	'Unnamed: 6']]

resumo_dic.columns = ['descricao', 'codigo', 'categoria', 'de']

# Lista de códigos a manter
codigos_manter = [
    'V0001', 'C006', 'C009', 'P00901', 'P018', 'P035', 'P03201', 'N00101','D00901', 'D014', 'E01201', 'E01601']

resumo_dic = resumo_dic[resumo_dic['codigo'].isin(codigos_manter)].reset_index(drop=True)


# Dicionário de renomeação
renomear_descricao = {
    'Unidade da Federação': 'estado',
    'Sexo': 'genero',
    'Cor ou raça': 'raca',
    'Qual foi o curso mais elevado que ___frequentou ': 'escolaridade',
    '___concluiu este curso que frequentou ' : 'concuiu_escolaridade',
    '...recebia/fazia normalmente nesse trabalho rendimento/retirada em dinheiro?' : 'renda_motorista',
    'Considerando saúde como um estado de bem-estar físico e mental, e não somente a ausência de doenças, como você avalia o seu estado de saúde? ': 'autoavaliacao_saude',
    'Em quantos dias da semana, o(a) Sr(a) costuma comer pelo menos um tipo de verdura ou legume (sem contar batata, mandioca, cará ou inhame) como alface, tomate, couve, cenoura, chuchu, berinjela, abobrinha? ': 'consumo_verduras',
    'Em quantos dias da semana o(a) Sr(a) costuma comer frutas? ': 'consumo_frutas',
    'Nos últimos trinta dias, o(a) Sr(a) chegou a consumir cinco ou mais doses de bebida alcoólica em uma única ocasião? ': 'consumo_alcool',
    'Quantos dias por semana o(a) Sr(a) costuma  (costumava)praticar exercício físico ou esporte?': 'pratica_ativ_fisica'
}

# Aplica a renomeação
resumo_dic['descricao'] = resumo_dic['descricao'].replace(renomear_descricao)
#resumo_dic = resumo_dic.dropna()


resumo_dic = resumo_dic[~(
    ((resumo_dic['descricao'] == 'raca') & (resumo_dic['codigo'] == 'C009')) &
    (resumo_dic['categoria'].isin(['5', '9']))
)].reset_index(drop=True)

resumo_dic[resumo_dic['descricao'] == 'raca']

resumo_dic = resumo_dic[resumo_dic['de'].str.strip().str.lower() != 'ignorado'].reset_index(drop=True)


resumo_dic

,descricao,codigo,categoria,de
0,estado,V0001,11,Rondônia
1,estado,V0001,12,Acre
2,estado,V0001,13,Amazonas
3,estado,V0001,14,Roraima
4,estado,V0001,15,Pará
5,estado,V0001,16,Amapá
6,estado,V0001,17,Tocantins
7,estado,V0001,21,Maranhão
8,estado,V0001,22,Piauí
9,estado,V0001,23,Ceará


In [47]:
# Vamos criar a estrutura solicitada:
# - Cada `descricao` (ex: estado, genero...) vira uma coluna
# - Abaixo, os valores da categoria (ex: 11, 12...) e a descrição (ex: Rondônia, Acre...)

# Filtra apenas linhas válidas com categoria e descrição
resumo_dic_validos = resumo_dic.dropna(subset=['categoria', 'de']).copy()

# Cria colunas nomeadas para pivotagem
resumo_dic_validos['coluna_descricao'] = resumo_dic_validos['descricao']
resumo_dic_validos['coluna_codigo'] = resumo_dic_validos['codigo']

# Cria uma coluna auxiliar para ordenar corretamente as linhas
resumo_dic_validos['ordem'] = resumo_dic_validos.groupby(['descricao', 'codigo']).cumcount()

# Pivotagem dos nomes (de)
pivot_nomes = resumo_dic_validos.pivot(index='ordem', columns='coluna_descricao', values='de')

# Pivotagem dos códigos (categoria)
pivot_codigos = resumo_dic_validos.pivot(index='ordem', columns='coluna_descricao', values='categoria')

# Renomeia colunas para indicar que são códigos
pivot_codigos.columns = [f"{col}_codigo" for col in pivot_codigos.columns]

# Junta os dois resultados
resultado_colunas_expandidas = pd.concat([pivot_nomes, pivot_codigos], axis=1)

resultado_colunas_expandidas.head(10)


,Qual era a ocupação (cargo ou função) que ___ tinha nesse trabalho,autoavaliacao_saude,concuiu_escolaridade,consumo_alcool,consumo_frutas,consumo_verduras,escolaridade,estado,genero,pratica_ativ_fisica,raca,renda_motorista,Qual era a ocupação (cargo ou função) que ___ tinha nesse trabalho_codigo,autoavaliacao_saude_codigo,concuiu_escolaridade_codigo,consumo_alcool_codigo,consumo_frutas_codigo,consumo_verduras_codigo,escolaridade_codigo,estado_codigo,genero_codigo,pratica_ativ_fisica_codigo,raca_codigo,renda_motorista_codigo
ordem,,,,,,,,,,,,,,,,,,,,,,,,
0,"Ver ""Classificação nacional de ocupações para pesquisas domiciliares (COD) 2010""",Muito boa,Sim,Sim,Dias,Dias,Creche,Rondônia,Homem,Dias,Branca,Valor em dinheiro,código,1,1,1,1 a 7,1 a 7,01,11,1,1 a 7,1,1
1,NaN,Boa,Não,Não,Nunca ou menos de uma vez por semana.,Nunca ou menos de uma vez por semana,Pré-escola,Acre,Mulher,Nunca ou menos de uma vez por semana,Preta,NaN,NaN,2,2,2,0,0,02,12,2,0,2,NaN
2,NaN,Regular,NaN,NaN,NaN,NaN,Classe de alfabetização – CA,Amazonas,NaN,NaN,Amarela,NaN,NaN,3,NaN,NaN,NaN,NaN,03,13,NaN,NaN,3,NaN
3,NaN,Ruim,NaN,NaN,NaN,NaN,Alfabetização de jovens e adultos,Roraima,NaN,NaN,Parda,NaN,NaN,4,NaN,NaN,NaN,NaN,04,14,NaN,NaN,4,NaN
4,NaN,Muito ruim,NaN,NaN,NaN,NaN,Antigo primário (elementar),Pará,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,NaN,NaN,05,15,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,Antigo ginasial (médio 1º ciclo),Amapá,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06,16,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,Regular do ensino fundamental ou do 1º grau,Tocantins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,07,17,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Educação de jovens e adultos (EJA) ou supletivo do ensino fundamental,Maranhão,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08,21,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,"Antigo científico, clássico etc. (médio 2º ciclo)",Piauí,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09,22,NaN,NaN,NaN,NaN


In [45]:
df_analises = pd.read_csv("data/df_analises.csv")

resumo_df_analises = df_analises[[
    'V0001', 'C006', 'C008','C009', 
    'P00901', 'P018', 'P035', 'P03201', 'N00101',
    'D00901', 'D014', 
    'E01201', 'E01601']]

resumo_df_analises.head() 

,V0001,C006,C008,C009,P00901,P018,P035,P03201,N00101,D00901,D014,E01201,E01601
0,11,2.0,55.0,1.0,1.0,3.0,1.0,2.0,3.0,5.0,1.0,5221.0,1.0
1,11,1.0,69.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,1.0,31.0,2.0,NaN,NaN,NaN,NaN,NaN,10.0,1.0,5246.0,1.0
3,11,1.0,9.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,2.0,6.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
# Vamos montar um código genérico que insere a categoria (nome) antes de cada coluna codificada


# Itera sobre as colunas codificadas presentes em resumo_df_analises
for codigo_var in resumo_dic['codigo'].unique():
    if codigo_var in resumo_df_analises.columns:
        descricao_var = resumo_dic.query(f"codigo == '{codigo_var}'")['descricao'].iloc[0]
        mapeamento = resumo_dic.query(f"codigo == '{codigo_var}'").set_index('categoria')['de'].to_dict()
        resumo_df_analises[codigo_var] = resumo_df_analises[codigo_var].astype(str)
        nova_coluna = resumo_df_analises[codigo_var].map(mapeamento)
        resumo_df_analises.insert(
            resumo_df_analises.columns.get_loc(codigo_var),
            descricao_var,
            nova_coluna
        )

resumo_df_analises.head()


/tmp/ipykernel_1119105/126456619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resumo_df_analises[codigo_var] = resumo_df_analises[codigo_var].astype(str)
/tmp/ipykernel_1119105/126456619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resumo_df_analises[codigo_var] = resumo_df_analises[codigo_var].astype(str)
/tmp/ipykernel_1119105/126456619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

,estado,V0001,genero,C006,C008,raca,C009,consumo_verduras,P00901,consumo_frutas,P018,pratica_ativ_fisica,P035,consumo_alcool,P03201,autoavaliacao_saude,N00101,escolaridade,D00901,concuiu_escolaridade,D014,Qual era a ocupação (cargo ou função) que ___ tinha nesse trabalho,E01201,renda_motorista,E01601
0,Rondônia,11,NaN,2.0,55.0,NaN,1.0,NaN,1.0,NaN,3.0,NaN,1.0,NaN,2.0,NaN,3.0,NaN,5.0,NaN,1.0,NaN,5221.0,NaN,1.0
1,Rondônia,11,NaN,1.0,69.0,NaN,4.0,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan
2,Rondônia,11,NaN,1.0,31.0,NaN,2.0,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,10.0,NaN,1.0,NaN,5246.0,NaN,1.0
3,Rondônia,11,NaN,1.0,9.0,NaN,2.0,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan
4,Rondônia,11,NaN,2.0,6.0,NaN,4.0,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan,NaN,nan


# motoristas_brasil

In [35]:
motoristas_brasil = pd.read_csv("data/dados_pns_2019_processado.csv")
motoristas_brasil

,Unnamed: 0,Estado,Expectativa_de_Vida,P00901,P018,P035,P03201,N00101
0,0,Acre,73.0,3.748588,3.282403,1.154556,1.864959,2.160954
1,1,Alagoas,71.6,4.016337,3.827324,1.364298,1.870901,2.262246
2,2,Amapá,73.9,3.933227,3.720827,1.689949,1.817703,2.239335
3,3,Amazonas,73.6,4.092107,3.574976,1.331450,1.871200,2.085123
4,4,Bahia,73.5,4.080068,3.900775,1.481649,1.807879,2.281865
5,5,Ceará,73.1,3.311669,3.953498,1.506315,1.859602,2.252431
6,6,Distrito Federal,77.6,5.039976,4.579721,2.000296,1.813148,1.975582
7,7,Espírito Santo,77.2,4.969262,3.786228,1.402282,1.825738,2.078004
8,8,Goiás,74.8,4.910139,3.709091,1.358644,1.827141,2.074273
9,9,Maranhão,71.7,3.304405,3.700798,1.310651,1.846407,2.288425


In [ ]:
import numpy as np
import pandas as pd

df_analises = motoristas_brasil.copy()
columns =  ["Expectativa_de_Vida", "P00901", "P018", "P035", "P03201", "N00101"]

DATA = df_analises[columns]

# Separar as variáveis independentes (X) e dependente (y)
X = DATA.iloc[:, 1:].values  # Matriz de variáveis independentes
y = DATA.iloc[:, 0].values  # Vetor de variável dependente
X_obs = DATA.iloc[0, 1:].values  # Observação específica de X
y_obs = DATA.iloc[0, 0]  # Observação específica de y (para comparação)

# Passo 1: Estimar as médias
mu_X = np.mean(X, axis=0)  # Média de X (vetor)
mu_Y = np.mean(y)  # Média de y (escalar)

# Passo 2: Estimar a matriz de covariância
# Combinar X e y em uma matriz única para calcular a covariância conjunta
data_combined = np.column_stack((X, y))  # Combinação de X e y
cov_matrix = np.cov(data_combined, rowvar=False)  # Matriz de covariância conjunta

# Separar os blocos da matriz de covariância
num_features = X.shape[1]
Sigma_X = cov_matrix[:num_features, :num_features]  # Matriz de covariância de X
Sigma_Y = cov_matrix[num_features:, num_features:]  # Variância de y (escalar)
Sigma_YX = cov_matrix[num_features:, :num_features]  # Covariância entre y e X
Sigma_XY = Sigma_YX.T  # Covariância entre X e y

# Passo 3: Calcular a média condicional
# Fórmula: μ_Y|X = μ_Y + Σ_YX Σ_X⁻¹ (X_obs - μ_X)
Sigma_X_inv = np.linalg.inv(Sigma_X)  # Inversa da matriz de covariância de X
mu_Y_given_X = mu_Y + Sigma_YX @ Sigma_X_inv @ (X_obs - mu_X)

# Passo 4: Calcular a variância condicional
# Fórmula: Var(Y|X) = Σ_Y - Σ_YX Σ_X⁻¹ Σ_XY
var_Y_given_X = Sigma_Y - Sigma_YX @ Sigma_X_inv @ Sigma_XY

# Resultados
print(f"Média condicional de y dado X_obs: {mu_Y_given_X}")
print(f"Variância condicional de y dado X_obs: {var_Y_given_X}")
print(f"Desvio condicional de y dado X_obs: {var_Y_given_X**0.5}")
print(f"Valor observado de y: {y_obs}")
print(f"Erro para 95% de nivel de confiança: {1.96 * var_Y_given_X**0.5} anos" )


# questionario

In [ ]:
questionario = pd.read_csv("data/Estilo de vida dos motoristas PX (respostas) - Respostas ao formulário 1.csv")

In [ ]:
# estado 
questionario = questionario.rename(columns={'Estado (UF) de origem ': 'Estado'})
questionario['Estado'] = questionario['Estado'].str.replace(r'\s*\(.*\)', '', regex=True).str.strip()

In [ ]:
# estudo

anos_estudo_mapping = {
    'Sem instrução (nunca frequentou a escola ou não concluiu nenhuma série)': 0,
    'Fundamental Incompleto': 4,
    'Fundamental Completo': 9,
    'Médio Incompleto': 10,
    'Médio Completo': 12,
    'Superior Incompleto': 14,
    'Superior Completo': 16}

questionario['anos_estudo'] = questionario['Escolaridade'].map(anos_estudo_mapping)
colunas = list(questionario.columns)
idx = colunas.index('Escolaridade')
colunas.insert(idx + 1, colunas.pop(colunas.index('anos_estudo')))
questionario = questionario[colunas] 

# Calcula o índice MYS com base na escala do PNUD (máximo = 15 anos)
questionario['indice_mys'] = questionario['anos_estudo'] / 15
questionario['indice_mys'] = questionario['indice_mys'].round(3)
colunas = list(questionario.columns)
idx = colunas.index('anos_estudo')
colunas.insert(idx + 1, colunas.pop(colunas.index('indice_mys')))
questionario = questionario[colunas]


# questionario

In [ ]:
# renda

questionario = questionario.rename(columns={'Informe sua renda média bruta nos últimos 3 meses considerando suas atividades como motorista de caminhão (em reais): ': 'renda_motorista',
                                            'Informe sua renda média bruta nos últimos 3 meses considerando todas as suas atividades profissionais (em reais): ': 'renda_total'})

def padronizar_renda(valor):
    if pd.isna(valor):
        return np.nan
    valor_str = str(valor).replace('.', '').replace(',', '').strip()
    try:
        valor_num = int(valor_str)
        # Se o número for muito pequeno (ex: 9 ou 80), provavelmente está em milhar
        if valor_num < 1000:
            return valor_num * 1000
        else:
            return valor_num
    except:
        return np.nan

colunas_renda = ['renda_motorista','renda_total']
for col in colunas_renda: questionario[col] = questionario[col].apply(padronizar_renda)



# Calcula os limites usando o método IQR (Interquartile Range)
q1 = questionario['renda_motorista'].quantile(0.25)
q3 = questionario['renda_motorista'].quantile(0.75)
iqr = q3 - q1

limite_inferior = q1 - 1.5 * iqr
limite_superior = q3 + 1.5 * iqr

questionario = questionario[
    (questionario['renda_motorista'] >= limite_inferior) &
    (questionario['renda_motorista'] <= limite_superior)]


questionario.head()

In [ ]:


# Define a renda mínima como valor fixo nacional
renda_min_fixa = 2416.45

# Função para aplicar por grupo de estado
def calcular_indice_renda_estado(grupo):
    renda_max_estado = grupo['renda_motorista'].max()
    
    # Aplica a fórmula do índice de renda com renda mínima fixa e máxima local
    grupo['indice_renda_estado'] = (
        np.log(grupo['renda_motorista']) - np.log(renda_min_fixa)
    ) / (
        np.log(renda_max_estado) - np.log(renda_min_fixa)
    )
    
    # Garante valores entre 0 e 1
    grupo['indice_renda_estado'] = grupo['indice_renda_estado'].clip(0, 1).round(3)
    
    return grupo

# Aplica o cálculo por estado
questionario = questionario.groupby('Estado', group_keys=False).apply(calcular_indice_renda_estado)

questionario

In [ ]:
# expectativa
# Dicionário único para os campos com a mesma escala (conforme arquivo simposio)
dias_semana_pontuacao = {
    '6 a 7 dias': 4,
    '4 a 5 dias': 3,
    '3 dias': 2,
    '1 a 2 dias': 1,
    'Nenhum dia': 0
}

# Dicionários individuais para os campos diferentes (conforme arquivo simposio)
pontuacoes_individuais = {
    'P03201': {
        'Sim': 0,
        'Não': 4
    },
    'N00101': {
        'Muito boa': 4,
        'Boa': 3,
        'Regular': 2,
        'Ruim': 1,
        'Muito Ruim': 0
    }
}

# Mapeamento dos nomes das colunas para os respectivos códigos
colunas_codigos = {
    'Quantos dias da semana consome verduras ou legumes?': 'P00901',
    'Quantos dias da semana consome frutas?': 'P018',
    'Quantos dias por semana você costuma praticar exercício físico ou esporte?': 'P035',
    'Nos últimos 30 dias, consumiu cinco ou mais doses de álcool em uma ocasião? (considerar uma dose: 350ml de cerveja; 150ml de vinho; ou 45ml destilado)': 'P03201',
    'Considerando saúde como um estado de bem-estar físico e mental, e não somente a ausência de doenças, como está sua saúde?': 'N00101'
}

# Aplica as pontuações e adiciona as colunas com os códigos
for pergunta, codigo in colunas_codigos.items():
    if codigo in ['P00901', 'P018', 'P035']:
        questionario[codigo] = questionario[pergunta].map(dias_semana_pontuacao)
    else:
        questionario[codigo] = questionario[pergunta].map(pontuacoes_individuais[codigo])

# Reorganiza para que as colunas com códigos fiquem ao lado das perguntas
colunas = list(questionario.columns)

for pergunta, codigo in colunas_codigos.items():
    if pergunta in colunas and codigo in colunas:
        idx_pergunta = colunas.index(pergunta)
        idx_codigo = colunas.index(codigo)
        # Remove o código da posição atual
        colunas.pop(idx_codigo)
        # Reinsere logo após a pergunta
        colunas.insert(idx_pergunta + 1, codigo)

# Reordena o DataFrame
questionario = questionario[colunas]

questionario.head(3)

In [ ]:
# Selecionar apenas as colunas desejadas
expectativa_px = questionario[['Estado', 'P00901', 'P018','P035', 'P03201', 'N00101','indice_mys']]
expectativa_px_mean = expectativa_px.groupby('Estado')[['P00901',	'P018',	'P035',	'P03201',	'N00101','indice_mys']].mean().round(2).reset_index()
expectativa_px_mean

In [ ]:
columns =  [ "P00901", "P018", "P035", "P03201", "N00101"]
DATA = expectativa_px_mean[columns]
X = DATA.iloc[:, 0:].values  # Matriz de variáveis independentes

conditional_sd = np.sqrt(var_Y_given_X)[0, 0]  # Desvio padrão condicional (escalares)
# Calcula a média condicional (expectativa de vida predita) para cada estado
predicted_y = []
for i in range(X.shape[0]):
    x_obs = X[i, :]
    # Fórmula: μ_Y|X = μ_Y + Σ_YX Σ_X⁻¹ (x_obs - μ_X)
    mu_Y_given_X = mu_Y + Sigma_YX @ Sigma_X_inv @ (x_obs - mu_X)
    predicted_y.append(mu_Y_given_X[0])  # Extraindo o valor escalar

# Adiciona os resultados ao DataFrame
expectativa_px_mean['Predicted_Expectativa_de_Vida'] = predicted_y
expectativa_px_mean['Predicted_Expectativa_de_Vida'] = expectativa_px_mean['Predicted_Expectativa_de_Vida'].round(1)
expectativa_px_mean['Conditional_SD'] = conditional_sd

# Exibe as colunas com os resultados
expectativa_px_mean